<a href="https://colab.research.google.com/github/ajai-cr7/DiskFailurePredictionDeepLearning/blob/main/DiskFailurePredictionImplementationFirstStage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [66]:
# Load Packages

import pandas as pd
import numpy as np
from time import time

from sklearn.ensemble import RandomForestClassifier
# from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.utils import resample
# from imblearn.over_sampling import SMOTE
from collections import Counter
from sklearn import metrics 
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import IsolationForest

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import make_scorer

In [67]:
# import necessary modules
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report

# load the data set
data = pd.read_csv('./2014-10-23.csv')

# print info about columns in the dataframe
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38592 entries, 0 to 38591
Data columns (total 85 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   date                  38592 non-null  object 
 1   serial_number         38592 non-null  object 
 2   model                 38592 non-null  object 
 3   capacity_bytes        38592 non-null  int64  
 4   failure               38592 non-null  int64  
 5   smart_1_normalized    38592 non-null  int64  
 6   smart_1_raw           38592 non-null  int64  
 7   smart_2_normalized    22335 non-null  float64
 8   smart_2_raw           22335 non-null  float64
 9   smart_3_normalized    38592 non-null  int64  
 10  smart_3_raw           38592 non-null  int64  
 11  smart_4_normalized    38592 non-null  int64  
 12  smart_4_raw           38592 non-null  int64  
 13  smart_5_normalized    38592 non-null  int64  
 14  smart_5_raw           38592 non-null  int64  
 15  smart_7_normalized 

In [68]:
data = data.drop(['serial_number','model','capacity_bytes'], axis = 1)

data['failure'].value_counts()
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38592 entries, 0 to 38591
Data columns (total 82 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   date                  38592 non-null  object 
 1   failure               38592 non-null  int64  
 2   smart_1_normalized    38592 non-null  int64  
 3   smart_1_raw           38592 non-null  int64  
 4   smart_2_normalized    22335 non-null  float64
 5   smart_2_raw           22335 non-null  float64
 6   smart_3_normalized    38592 non-null  int64  
 7   smart_3_raw           38592 non-null  int64  
 8   smart_4_normalized    38592 non-null  int64  
 9   smart_4_raw           38592 non-null  int64  
 10  smart_5_normalized    38592 non-null  int64  
 11  smart_5_raw           38592 non-null  int64  
 12  smart_7_normalized    38592 non-null  int64  
 13  smart_7_raw           38592 non-null  int64  
 14  smart_8_normalized    22338 non-null  float64
 15  smart_8_raw        

In [69]:
data.fillna(data.mean(), inplace=True)

# data = data[["failure","smart_1_normalized","smart_2_normalized","smart_3_normalized","smart_4_normalized"]]

#take first 100000 rows with 5 column attributes
data = data.loc[1:38000, ["smart_1_normalized","smart_3_normalized","smart_4_normalized","smart_5_normalized","failure","date"]]

#change float to int
data["smart_1_normalized"] = data["smart_1_normalized"].fillna(0.0).astype(int)
data["smart_3_normalized"] = data["smart_3_normalized"].fillna(0.0).astype(int)
data["smart_4_normalized"] = data["smart_4_normalized"].fillna(0.0).astype(int)
data["smart_5_normalized"] = data["smart_5_normalized"].fillna(0.0).astype(int)

print(data)

#print how many nan values present 
# print(data["failure"].isnull().sum())

#display how much data is failure or success
data['failure'].value_counts()

       smart_1_normalized  smart_3_normalized  ...  failure        date
1                     100                 127  ...        0  2014-10-23
2                     100                 124  ...        0  2014-10-23
3                     100                 137  ...        0  2014-10-23
4                     200                 195  ...        0  2014-10-23
5                     100                 131  ...        0  2014-10-23
...                   ...                 ...  ...      ...         ...
37996                 118                  97  ...        0  2014-10-23
37997                 100                 124  ...        0  2014-10-23
37998                 100                 126  ...        0  2014-10-23
37999                 100                 100  ...        0  2014-10-23
38000                 105                  94  ...        0  2014-10-23

[38000 rows x 6 columns]


0    37992
1        8
Name: failure, dtype: int64

In [70]:

def grid_tune_isolationforest():
    iso = IsolationForest(random_state=0)
    # Number of trees in random forest
    n_estimators = [int(x) for x in np.linspace(start = 100, stop = 200, num = 5)]
    # The number of samples to draw from X to train each base estimator.
    max_samples = [0.2, 0.5, 0.8, 1]
    # the proportion of outliers in the data set
    contamination = ['auto', 0.1, 0.2, 0.3]
    # The number of features to draw from X to train each base estimator.
    max_features = [0.2, 0.5, 0.8, 1]
    # Method of selecting samples for training each tree
    bootstrap = [True, False]

    
    random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'bootstrap': bootstrap,
               'max_samples': max_samples,
               'contamination': contamination
                }
    def acc(y_true, y_pred): 
        y_pred = [0 if x > 0 else 1 for x in y_pred]
        return accuracy_score(y_true, y_pred)
    def f1(y_true, y_pred): 
        y_pred = [0 if x > 0 else 1 for x in y_pred]
        return f1_score(y_true, y_pred)
    iso_grid = GridSearchCV(
        estimator = iso, 
        param_grid = random_grid, 
        cv = 3, 
        verbose=3, 
        n_jobs = -1, 
        scoring = {'f1': make_scorer(f1), 'acc': make_scorer(acc)}, 
        refit="f1",
        return_train_score=True
    )
    
    return iso_grid

In [80]:

# Reading data from a combined file with both good and failed drives
def split_train_val_test_data():
    
    df = data
    print(df)

    resample_data = True
    smote_data = False

    df_good = df.loc[df['failure'] == 0]
    df_bad = df.loc[df['failure'] == 1]
     
    df_good = df_good.sort_values(["date"])
    df_bad = df_bad.sort_values(["date"])

    good_y = df_good["failure"]
    bad_y = df_bad["failure"]

    # Split into train (80%) and test (20%)
    X_train_good, X_test_good, y_train_good, y_test_good = train_test_split(
        df_good, good_y, train_size=0.8, shuffle=False)
    X_train_bad, X_test_bad, y_train_bad, y_test_bad = train_test_split(
        df_bad, bad_y, train_size=0.8, shuffle=False)


    # Split train into train and validation
    # Train(60%), Val(20%), Test(20%)
#     X_train_good, X_val_good, y_train_good, y_val_good = train_test_split(
#         X_train_good, y_train_good, train_size=0.75, shuffle=False)
#     X_train_bad, X_val_bad, y_train_bad, y_val_bad = train_test_split(
#         X_train_bad, y_train_bad, train_size=0.75, shuffle=False)
        
    if resample_data:
        X_train_bad = resample(df_bad, replace=True, n_samples=len(X_train_good), random_state=1)
        X_train_bad = X_train_bad.sort_values(["date"])

    y_train_bad = X_train_bad["failure"]

    X_train = pd.concat([X_train_good, X_train_bad], axis=0)
    y_train = pd.concat([y_train_good, y_train_bad], axis=0)
#     X_val = pd.concat([X_val_good, X_val_bad], axis=0)
#     y_val = pd.concat([y_val_good, y_val_bad], axis=0)
    X_test = pd.concat([X_test_good, X_test_bad], axis=0)
    y_test = pd.concat([y_test_good, y_test_bad], axis=0)

    ignore_cols = ["date"] 

    X_train.drop(columns=ignore_cols, inplace=True, axis=1)
    # X_val.drop(columns="date", inplace=True, axis=1)
    X_test.drop(columns=ignore_cols, inplace=True, axis=1)

    if smote_data:
        sm = SMOTE(random_state=42)
        X_train, y_train = sm.fit_resample(X_train, y_train)

    #return (X_train, X_val, X_test, y_train, y_val, y_test)
    return (X_train, X_test, y_train, y_test)

In [55]:
def run(models, tune_model=False):
    #X_train, X_val, X_test, y_train, y_val, y_test = split_train_val_test_data(drive_file = "/ST12000NM0007_last_day_all_q_raw.csv", smote_data=True)
    X_train, X_test, y_train, y_test = split_train_val_test_data()
    print("Data loaded successfully...\n")
    for model in models:  
        print("\n\n *", type(model).__name__)  
        
        # if(type(model).__name__ == "XGBClassifier" and tune_model):
        #     tune_xgb(model, X_train, y_train)

        start = time()
        model.fit(X_train, y_train)
        end = time()
        print("\nTime to train:", str((end - start)/60), " mins")
        
        print("Best Parameter", model.best_params_)
        # Test set results
        print("\n- Results on test set: ")
        y_pred = model.predict(X_test)
        y_pred = y_pred = [0 if x > 0 else 1 for x in y_pred]
        print("Accuracy: ", accuracy_score(y_test, y_pred))
        print("Scores:\n", classification_report(y_test, y_pred))

In [ ]:
# # load the data set
# data = pd.read_csv('./2021-01-02.csv')

# # print info about columns in the dataframe
# print(data.info())


if __name__ == "__main__":
    models_list = []
    iso = grid_tune_isolationforest()
    print(iso)
    models_list.append(iso)
    run(models_list,tune_model=True)
    

GridSearchCV(cv=3, error_score=nan,
             estimator=IsolationForest(behaviour='deprecated', bootstrap=False,
                                       contamination='auto', max_features=1.0,
                                       max_samples='auto', n_estimators=100,
                                       n_jobs=None, random_state=0, verbose=0,
                                       warm_start=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'bootstrap': [True, False],
                         'contamination': ['auto', 0.1, 0.2, 0.3],
                         'max_features': [0.2, 0.5, 0.8, 1],
                         'max_samples': [0.2, 0.5, 0.8, 1],
                         'n_estimators': [100, 125, 150, 175, 200]},
             pre_dispatch='2*n_jobs', refit='f1', return_train_score=True,
             scoring={'acc': make_scorer(acc), 'f1': make_scorer(f1)},
             verbose=3)
       smart_1_normalized  smart_3_normalized  ...  failure        da

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:   53.7s
[Parallel(n_jobs=-1)]: Done 124 tasks      | elapsed:  4.0min
[Parallel(n_jobs=-1)]: Done 284 tasks      | elapsed: 10.3min
